<a href="https://colab.research.google.com/github/nikhilmeghnani/deltalake/blob/main/delta_lake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz
!tar xf spark-3.0.3-bin-hadoop3.2.tgz
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:0.7.0 --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell'

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('delta_test').getOrCreate()

In [ ]:
history = spark.range(1,11)
history = history.withColumn("type", lit("history"))
history.show()

+---+-------+
| id|   type|
+---+-------+
|  1|history|
|  2|history|
|  3|history|
|  4|history|
|  5|history|
|  6|history|
|  7|history|
|  8|history|
|  9|history|
| 10|history|
+---+-------+



In [ ]:
history.write.format("delta").save("delta_sample2")

In [ ]:
spark.sql("describe detail 'delta_sample2'").show(truncate=False)

+------+------------------------------------+----+-----------+---------------------------+-----------------------+-----------------------+----------------+--------+-----------+----------+----------------+----------------+
|format|id                                  |name|description|location                   |createdAt              |lastModified           |partitionColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|
+------+------------------------------------+----+-----------+---------------------------+-----------------------+-----------------------+----------------+--------+-----------+----------+----------------+----------------+
|delta |62396424-eadd-41af-bac2-6bc73b03c8d8|null|null       |file:/content/delta_sample2|2021-12-03 08:49:37.532|2021-12-03 08:49:39.355|[]              |2       |1494       |[]        |1               |2               |
+------+------------------------------------+----+-----------+---------------------------+----------------------

In [ ]:
# spark.sql("select id, type from delta_sample2").show()
history.write.format("delta").saveAsTable("delta_sample2_table")

In [ ]:
spark.sql("select * from delta_sample2_table").show()

+---+-------+
| id|   type|
+---+-------+
|  1|history|
|  2|history|
|  3|history|
|  4|history|
|  5|history|
|  6|history|
|  7|history|
|  8|history|
|  9|history|
| 10|history|
+---+-------+



# New Section

In [ ]:
!ls -l delta_sample2

total 12
drwxr-xr-x 2 root root 4096 Nov 19 07:28 _delta_log
-rw-r--r-- 1 root root  748 Nov 19 07:28 part-00000-3da36727-e770-41a9-a6f4-9a02b49f02bf-c000.snappy.parquet
-rw-r--r-- 1 root root  746 Nov 19 07:28 part-00001-c17a51e7-2485-4e4f-85c1-b980d4ed4030-c000.snappy.parquet


In [ ]:
updates = spark.range(6,11)
updates = updates.withColumn("type", lit("updates"))
updates.show()

+---+-------+
| id|   type|
+---+-------+
|  6|updates|
|  7|updates|
|  8|updates|
|  9|updates|
| 10|updates|
+---+-------+



In [ ]:
from delta.tables import *
deltaTable = DeltaTable.forPath(spark, "delta_sample2")
deltaTable.alias("history").merge(
    updates.alias("updates"),
    "history.id = updates.id").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

In [ ]:
deltaTable.history().show(truncate=False)

+-------+-----------------------+------+--------+---------+--------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|version|timestamp              |userId|userName|operation|operationParameters                         |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                                                                                          |userMetadata|
+-------+-----------------------+------+--------+---------+--------------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------

In [ ]:
spark.sql("describe detail 'delta_sample2'").show(truncate=False)

+------+------------------------------------+----+-----------+---------------------------+-----------------------+----------------------+----------------+--------+-----------+----------+----------------+----------------+
|format|id                                  |name|description|location                   |createdAt              |lastModified          |partitionColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|
+------+------------------------------------+----+-----------+---------------------------+-----------------------+----------------------+----------------+--------+-----------+----------+----------------+----------------+
|delta |4a5a13ee-47ef-4a44-adc8-19862429d729|null|null       |file:/content/delta_sample2|2021-11-19 07:28:53.221|2021-11-19 07:29:38.46|[]              |7       |4623       |[]        |1               |2               |
+------+------------------------------------+----+-----------+---------------------------+-----------------------+--

In [ ]:
spark.read.format("delta").load("delta_sample2").groupBy("type").count().show()

+-------+-----+
|   type|count|
+-------+-----+
|updates|    5|
|history|    5|
+-------+-----+



In [ ]:
spark.sql("SELECT * FROM delta_sample2 VERSION AS OF 0").show()
spark.sql("SELECT COUNT(*) FROM delta_sample2 VERSION AS OF 0").show()

ParseException: ignored

In [ ]:
df = spark.read.csv("/content/test.csv")
df.show()
# spark.sql("create table california_housing_train(longitude string,latitude string, housing_median_age double, total_rooms double, population double,population double,households string,median_income string  )  /content/sample_data/california_housing_train.csv")

+--------+---+------+------+---------+
|     _c0|_c1|   _c2|   _c3|      _c4|
+--------+---+------+------+---------+
|    name|age|gender| phone|code_type|
|    john| 25|     m| 12356|        1|
|    john| 25|     m| 12356|        1|
|superman| 26|     m|412356|        1|
|   peter| 28|     m|789456|        2|
+--------+---+------+------+---------+



In [ ]:
spark.sql("create table test1 using DELTA location '/content/test.csv'")

AnalysisException: ignored